# Visualizing Topic Model

This is a work-in-progress Notebook providing different types of visualization for a topic model. The first visualization (`pyLDAvis`) allows a researcher to interact with the *vocabulary* of the corpus and its distribution over topics. pyLDAvis is an out-of-the-box package, a wrapper around the R-library `LDAvis`.

The second visualization employs the `bokeh` library and allows a researcher to interact with (and inspect) *documents* and the distribution of topics over documents. 

The data used in this notebook are scraped from [ORACC](http://oracc.org). Most of the techniques used here, however, may be applied to any set of documents.

# Dependencies and Versions
# TODO: rewrite

pyLDAvis 2.0 is incompatible with Pandas 0.19 (check for pyLDAvis 2.1.0 or later). The MDS and tSNE computations necessary for plotting the distribution of documents in the topic model need scikit-learn 0.18. This notebook was written for Python 3.5.

In [1]:
import pandas as pd
import glob
import numpy as np

# Read in Data
First read the directory with the relevant texts. These files contain lemmatization in ORACC style, e.g. `lugal[king]N`. The documents list lemmatizations per line.
# TODO: acquire data word-by-word

In [2]:
path =r'data/' # use your path
allFiles = glob.glob(path +"saao" + "*.txt")
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
data = pd.concat(list_)
data.head()

,id_text,text_name,version,l_no,text
0,saao/saa15/P313978,SAA 15 296. Sending People to the King (CT 53 ...,SAA 15 296. Sending People to the King (CT 53 ...,o 2',ša-qurbūti[close-follower]N
1,saao/saa15/P313978,SAA 15 296. Sending People to the King (CT 53 ...,SAA 15 296. Sending People to the King (CT 53 ...,o 4',annûri[now]AV
2,saao/saa15/P313978,SAA 15 296. Sending People to the King (CT 53 ...,SAA 15 296. Sending People to the King (CT 53 ...,o 5',ana[to]PRP šarru[king]N
3,saao/saa15/P313978,SAA 15 296. Sending People to the King (CT 53 ...,SAA 15 296. Sending People to the King (CT 53 ...,o 6',ša-qurbūti[close-follower]N
0,saao/saa06/P335129,SAA 06 127. Atuehu 'Third Man' Buys a Slave (6...,SAA 06 127. Atuehu 'Third Man' Buys a Slave (6...,o 1,kūm[instead-of]PRP kunukku[seal]N ṣupru[nail]N


# Collapse lines to one row per document
In order to transform this DataFrame into a proper input for topic modeling we need to discard the columns `version` and `l_no` and concatenate all the text that belongs to one document in a single row. Some lines have no content in the `text` column - these lines need to be dropped.

First select the relevant columns and drop the rows that have no text content.

# TODO: transform word-by-word to list

In [3]:
data = data[['id_text', 'text_name', 'text']]
data = data.dropna()
data.head()

,id_text,text_name,text
0,saao/saa15/P313978,SAA 15 296. Sending People to the King (CT 53 ...,ša-qurbūti[close-follower]N
1,saao/saa15/P313978,SAA 15 296. Sending People to the King (CT 53 ...,annûri[now]AV
2,saao/saa15/P313978,SAA 15 296. Sending People to the King (CT 53 ...,ana[to]PRP šarru[king]N
3,saao/saa15/P313978,SAA 15 296. Sending People to the King (CT 53 ...,ša-qurbūti[close-follower]N
0,saao/saa06/P335129,SAA 06 127. Atuehu 'Third Man' Buys a Slave (6...,kūm[instead-of]PRP kunukku[seal]N ṣupru[nail]N


Group the rows by `id_text` and apply the `join` function to the `text` column. Transform the aggregated data into a new DataFrame.

In [36]:
docs = data['text'].groupby([data['id_text']]).apply(' '.join)
df = pd.DataFrame(docs)
df.head()

,text
id_text,
saao/saa01/P224485,awātu[word]N šarru[king]N ana[to]PRP Aššur-šar...
saao/saa01/P313416,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313417,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313425,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313427,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...


In [37]:
ids_names = data[['id_text','text_name']].drop_duplicates().set_index('id_text')
df = pd.merge(df, ids_names, right_index=True, left_index=True)
df.head()

,text,text_name
id_text,,
saao/saa01/P224485,awātu[word]N šarru[king]N ana[to]PRP Aššur-šar...,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...
saao/saa01/P313416,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...,SAA 01 158. Gold and Silver Objects Sent to th...
saao/saa01/P313417,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...,SAA 01 233. More Land to Bel-duri (CT 53 002)
saao/saa01/P313425,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...,SAA 01 179. No Iron to the Arabs! (CT 53 010)
saao/saa01/P313427,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...,SAA 01 152. The Affair of Gidgidanu and His Br...


In [38]:
from gensim import corpora, models, utils
import gensim

# Read in the texts
[For test purposes one may select only the first 100 documents. Remove the hashmark (#) from the first line of the following cell if you wish to do that]

In [39]:
#df = df[:100]
texts = df['text']

# Tokenize
Transform the text data into a list of lists.
# TODO: no tokenization necessary if acquired as list

In [40]:
texts = [t.lower().split() for t in texts]

# POS-filter
The variable `posfilter` holds the last two characters of lemmatized words with allowed Part of Speech tags. If, for instance, you wish to select Verbs, Adjectives, and Nouns (in Akkadian), posfilter will be `']n', 'aj', ']v']`. Note that one-character pos-tags need the right bracket!
The POS labels are:
* "n", #Nouns
* "v", #Verbs
* "aj", #Adjectives
* "av", #Adverbs
* "an", #Agricultural Name
* "cn", #Celestial Name
* "dn", #Divine Name
* "en", #Ethnicity Name
* "fn", #Field Name
* "gn", #Geographical Name (lands, etc.)
* "ln", #Lineage Name (ancestral clan)
* "mn", #Month Name
* "on", #Object Name
* "pn", #Personal Name
* "qn", #Quarter (of a city) Name
* "rn", #Royal Name
* "sn", #Settlement Name
* "tn", #Temple Name
* "wn", #Watercourse Name
* "yn", #Year Name
* "nu", #Numeral


In [41]:
posfilter = [']n', ']v', 'aj']
#include nouns, verbs, and adjectives, not numerals, prepositions or proper nouns
texts = [[word for word in text if word[-2:] in posfilter] for text in texts]

# Stop words

Stop words are very frequent words that are not able to distinguish between topics. This includes, for instance, prepositions - but can also be filtered out by the POS filter. The following nouns and verbs are too frequent to contribute to the analysis.
# TODO
stoplist may not be necessary with dictionary.filter_extremes

In [42]:
stoplist = [
'šarru[king]n',
'bēlu[lord]n',
'libbu[interior]n',
'muhhu[skull]n',
'ardu[slave]n',
'šulmu[completeness]n',
'šapāru[send]v',
'alāku[go]v',
'qabû[say]v',
'pānu[front]n',
'māru[son]n',
'bītu[house]n',
'epēšu[do]v',
'wabālu[bring]v',
'šakānu[put]v',
'amāru[see]v',
'bašû[exist]v',
'našû[lift]v',
'izuzzu[stand]v',
'ūmu[day]n',
'ṭābu[good]aj',
'mādu[many]aj',
'nadānu[give]v',
'tadānu[give]v',
'ṣehru[small]aj',
'mimmû[all]n',
'gimru[totality]n',
'gabbu[totality]n',
'šâlu[ask]v',
'šemû[hear]v',
'ūmu[day]n',
'awātu[word]n',
'erēbu[enter]v'
]
texts = [[word for word in text if word not in stoplist] for text in texts]

# Filter out texts that have too few words left
Identify texts that have less than 10 lemmas left and use that selection for the list `texts` and for the dataframe `df`. 

In [43]:
minimum = 10
bo = [len(text) >= minimum for text in texts]
df = df[bo]
texts = [texts[i] for i in range(0, len(texts)) if bo[i]]

How many documents did we start with, and how many do we have left?

In [44]:
len(bo), len(texts)

(3230, 1754)

# Dictionary
create the gensim Dictionary and filter for words that are too common or too rare (no_above may be set too low here).

In [45]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
## CHECK - is this done correctly?

In [46]:
corpus = [dictionary.doc2bow(doc) for doc in texts]

# Compute the Model

Set the seed, indicate the number of topics and run the model

In [58]:
seed = 25
np.random.seed(seed)
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
ntopics = 10
# Running and Training LDA model on the document term matrix.
ldamodel = Lda(corpus, num_topics=ntopics, id2word = dictionary, passes=50)

List the top words and their probabilities in all topics. Note: the topic numbers here are not the ones used below in the visualizations! (The topics are the same, but not their numbers).

In [59]:
ldamodel.show_topics(ntopics, formatted = False)

[(0,
  [('ṭūbu[goodness]n', 0.03979085),
   ('ahu[brother]n', 0.03961113),
   ('šīru[flesh]n', 0.021492345),
   ('karābu[pray]v', 0.02087048),
   ('mātu[land]n', 0.015440333),
   ('abu[father]n', 0.014686441),
   ('ṭuppu[tablet]n', 0.013939629),
   ('ṭēmu[(fore)thought]n', 0.013369521),
   ('šattu[year]n', 0.012945537),
   ('mahru[received]aj', 0.012873382)]),
 (1,
  [('šību[witness]n', 0.10909382),
   ('dīnu[legal-decision]n', 0.06037998),
   ('manû[unit]n', 0.060082406),
   ('dabābu[speak]v', 0.057204586),
   ('kaspu[silver]n', 0.044240106),
   ('ṣarpu[silver]n', 0.041199744),
   ('târu[turn]v', 0.040347576),
   ('leqû[take]v', 0.038155682),
   ('tadnu[given]aj', 0.024655754),
   ('gammuru[finished]aj', 0.024315031)]),
 (2,
  [('šipru[sending]n', 0.031405635),
   ('ṣābu[people]n', 0.025785116),
   ('dullu[trouble]n', 0.02386924),
   ('mātu[land]n', 0.02158664),
   ('ṭēmu[(fore)thought]n', 0.01981744),
   ('sahāru[go-around]v', 0.019573363),
   ('dabābu[speak]v', 0.01760037),
   ('ēka

# pyLDAvis
Use pyLDAvis to visualize the topic model. By default, pyLDAvis will order the topics by [prevalence](https://github.com/bmabey/pyLDAvis/issues/59) (topic 1 is the most prevalent topic). That means that the topic numbers in the visualization do not agree with the topic numbers in the lda model. To prevent this behaviour one may use `sort_topics=False` in the `prepare` command. The advantage of ordering the topics by prevalence, however, is that new instances of the lda model are more comparable (that is, the same topic will receive the same number). Note that the library was written in Java for R, and so the numbering in the visualization begins with 1 (not with 0). The topic numbers in the Document/Topic and Topic/Term matrices below will be adjusted to be compatible with the pyLDAvis visualization.

PyLDAvis needs a large output box. The `%%html` lines below create such a box (for the code see [here](http://stackoverflow.com/questions/18770504/resize-ipython-notebook-output-window)). 

%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>


In [60]:
import os
import pyLDAvis.gensim
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
if not os.path.exists('vis'):
    os.makedirs('vis')
pyLDAvis.save_html(vis, 'vis/lda_terms.html')
pyLDAvis.display(vis)

# Document/Topic Probability
The function `get_document_topics()` will list the probability of the topics in a single document. In order to get all the topics set the argument `minimum_probability` to zero. 

In [61]:
ldamodel.get_document_topics(corpus[1], minimum_probability=0)

[(0, 0.0028580185),
 (1, 0.24180739),
 (2, 0.0028579903),
 (3, 0.00285852),
 (4, 0.0028580145),
 (5, 0.73532766),
 (6, 0.002858067),
 (7, 0.0028580234),
 (8, 0.0028582057),
 (9, 0.0028581412)]

# Create Document/Topic Probability Table
In order to create a full Document/Topic probability table we iterate over the entire corpus with the `get_document_topics()` function. This creates a list of list (`list_of_doctopics`) where each list represents the probability of each topic in a document. The probability is represented in a tuple (topic_number, probability). The `list_of_probabilities` preserves only the probabilities. This list of lists is transformed into a DataFrame, whith as index the index of the original DataFrame with the tokenized data.  

In [62]:
list_of_doctopics = [ldamodel.get_document_topics(corpus[i], minimum_probability=0) for i in range(len(corpus))]
list_of_probabilities = [[probability for label,probability in distribution] for distribution in list_of_doctopics]
d_t_df = pd.DataFrame(list_of_probabilities)
d_t_df = d_t_df.set_index(df.index)
d_t_df.head()

,0,1,2,3,4,5,6,7,8,9
id_text,,,,,,,,,,
saao/saa01/P224485,0.001177,0.001177,0.989409,0.001177,0.001177,0.001177,0.001177,0.001177,0.001177,0.001177
saao/saa01/P313416,0.002858,0.241826,0.002858,0.002859,0.002858,0.735309,0.002858,0.002858,0.002858,0.002858
saao/saa01/P313417,0.002223,0.002223,0.002223,0.267081,0.002223,0.002223,0.715137,0.002223,0.002223,0.002223
saao/saa01/P313425,0.001352,0.233777,0.261452,0.001352,0.116486,0.001352,0.133174,0.142918,0.058904,0.049232
saao/saa01/P313427,0.640715,0.004167,0.004168,0.004167,0.325942,0.004168,0.004168,0.004168,0.004167,0.004168


# Renumber Topics
Rename the topics `topic_1` to `topic_n` in accordance with the pyLDAvis visualization and add `text_name`.

In [63]:
topics = ['topic_' + str(i+1) for i in range(ntopics)]
d_t_df.columns = topics
#d_t_df['text_name'] = df['text_name']
d_t_df

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10
id_text,,,,,,,,,,
saao/saa01/P224485,0.001177,0.001177,0.989409,0.001177,0.001177,0.001177,0.001177,0.001177,0.001177,0.001177
saao/saa01/P313416,0.002858,0.241826,0.002858,0.002859,0.002858,0.735309,0.002858,0.002858,0.002858,0.002858
saao/saa01/P313417,0.002223,0.002223,0.002223,0.267081,0.002223,0.002223,0.715137,0.002223,0.002223,0.002223
saao/saa01/P313425,0.001352,0.233777,0.261452,0.001352,0.116486,0.001352,0.133174,0.142918,0.058904,0.049232
saao/saa01/P313427,0.640715,0.004167,0.004168,0.004167,0.325942,0.004168,0.004168,0.004168,0.004167,0.004168
...,...,...,...,...,...,...,...,...,...,...
saao/saa19/P393696,0.008335,0.008335,0.268400,0.008335,0.008336,0.008336,0.008336,0.350868,0.322386,0.008335
saao/saa19/P393702,0.006670,0.006669,0.329329,0.006671,0.006668,0.006668,0.200115,0.423872,0.006669,0.006668
saao/saa19/P393703,0.006253,0.250063,0.006253,0.006254,0.006253,0.006253,0.251817,0.454347,0.006255,0.006253


# Create Topic / Term table
This is a table with N rows (the number of topics) and M columns (the number of individual terms in the Dictionary). The table indicates the probability of each term in each topic.

In [64]:
topic_term = ldamodel.show_topics(ntopics, formatted=False, num_words=len(dictionary))

The object `topic_term` is a list of list. Each topic is represented by a list of tuples in the form `(word, probability)`. The following code pulls out the probabilities for each word in each topic (`topic_term[i][1]`) and creates a list of DataFrames with the words as index (rows) and the probabilities as the only column. The DataFrames are concatenated into a single DataFrame. 

In [65]:
topic_term_list = [pd.DataFrame(topic_term[i][1]).set_index(0) for i in range(0, ntopics)]
t_t_df_ = pd.concat(topic_term_list, axis=1, ignore_index=True, sort=True)
t_t_df_.head()

,0,1,2,3,4,5,6,7,8,9
abati[(meaning-unknown)]n,0.000032,0.000100,0.000015,0.000014,0.000031,0.000017,0.000023,0.000049,0.000035,0.000034
abbūtu[fatherhood]n,0.000032,0.000009,0.000015,0.000014,0.000406,0.001120,0.001429,0.000049,0.000038,0.000490
abiktu[defeat]n,0.000032,0.000009,0.000162,0.000014,0.000031,0.000191,0.000255,0.000049,0.000035,0.000034
abku[captive]n,0.000032,0.000282,0.000015,0.000014,0.000031,0.000017,0.000023,0.000049,0.000035,0.000034
abnu[stone]n,0.000707,0.000105,0.000015,0.000014,0.000031,0.001722,0.000257,0.000049,0.000035,0.000034


Rename the columns (`0` becomes `topic_1` etc) and Transpose to Topic/Term matrix.

In [66]:
t_t_df_.columns = topics
t_t_df = t_t_df_.T
t_t_df

,abati[(meaning-unknown)]n,abbūtu[fatherhood]n,abiktu[defeat]n,abku[captive]n,abnu[stone]n,abu[father]n,abullu[(city)-gate]n,abāku[lead-away]v,abālu[dry-(up)]v,abāru[(the-metal)-lead]n,...,ṭābtu[salt]n,ṭābtānu[doer-of-good]n,ṭābtūtu[goodwill]n,ṭātu[bribe]n,ṭēhu[immediate-vicinity]n,ṭēmu[(fore)thought]n,ṭēmūtu[of-order]n,ṭīdu[clay]n,ṭīmu[yarn]n,ṭūbu[goodness]n
topic_1,0.000032,0.000032,0.000032,0.000032,0.000707,0.014686,0.000420,0.007912,0.000351,0.000032,...,0.000032,0.000670,0.000032,0.000445,0.000032,0.013370,0.000032,0.000032,0.000032,0.039791
topic_2,0.000100,0.000009,0.000009,0.000282,0.000105,0.000153,0.000248,0.000009,0.000009,0.000009,...,0.000009,0.000009,0.000009,0.000009,0.000101,0.000236,0.000009,0.000009,0.000009,0.000009
topic_3,0.000015,0.000015,0.000162,0.000015,0.000015,0.003825,0.000015,0.001546,0.000162,0.000015,...,0.000015,0.000015,0.000015,0.000015,0.000015,0.019817,0.000175,0.000015,0.000015,0.000019
topic_4,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,...,0.000014,0.000014,0.000014,0.000014,0.000014,0.000529,0.000014,0.000014,0.000014,0.000014
topic_5,0.000031,0.000406,0.000031,0.000031,0.000031,0.003788,0.009173,0.000607,0.000031,0.000031,...,0.000031,0.000031,0.000031,0.000031,0.000031,0.000826,0.000031,0.000640,0.000031,0.001861
topic_6,0.000017,0.001120,0.000191,0.000017,0.001722,0.020500,0.000017,0.000535,0.000017,0.000191,...,0.000017,0.000017,0.000017,0.000831,0.000017,0.008136,0.000017,0.000017,0.000017,0.005814
topic_7,0.000023,0.001429,0.000255,0.000023,0.000257,0.000377,0.000282,0.001376,0.000023,0.000023,...,0.000023,0.000023,0.000254,0.000023,0.000023,0.034110,0.000023,0.000023,0.000023,0.000023
topic_8,0.000049,0.000049,0.000049,0.000049,0.000049,0.007496,0.001064,0.000050,0.000049,0.000049,...,0.000049,0.000049,0.000049,0.000049,0.000049,0.008326,0.000049,0.000049,0.000049,0.000049
topic_9,0.000035,0.000038,0.000035,0.000035,0.000035,0.005836,0.000035,0.001198,0.000035,0.000035,...,0.000035,0.000035,0.000035,0.000393,0.000035,0.003450,0.000357,0.000035,0.000388,0.000036
topic_10,0.000034,0.000490,0.000034,0.000034,0.000034,0.004819,0.000034,0.000694,0.000034,0.000034,...,0.000373,0.000034,0.000034,0.000034,0.000034,0.003047,0.000034,0.000034,0.000034,0.006482


In [67]:
#just checking
t_t_df['ēkallu[palace]n']

topic_1     0.006836
topic_2     0.003017
topic_3     0.016681
topic_4     0.004711
topic_5     0.011188
topic_6     0.004934
topic_7     0.009549
topic_8     0.006105
topic_9     0.023207
topic_10    0.001830
Name: ēkallu[palace]n, dtype: float64

# Visualize the Documents
While pyLDAvis is an excellent tool for exploring the topic/term aspect of a topic model (the words and their probabilities in each topic) it does not provide access to the document/topic aspect (the probability distribution of topics in each document). The visualization below plots all the documents according to their (cosine) distances in the Document/Term DataFrame. Each document (data point in the visualization) is colored according to the most prevalent topic and the size of the dot represents the probability of the most prevalent topic in that document.

In [68]:
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer

Compute the distances between each of the documents. Use either the Document/Topic Dataframe or the Document/Term Dataframe (constructed below) to measure distance.

In [69]:
cv = CountVectorizer(analyzer='word', token_pattern=r'[^ ]+', vocabulary=list(t_t_df.columns.values))
dtm = cv.fit_transform(df['text'])
dtm_df = pd.DataFrame(dtm.toarray(), columns = cv.get_feature_names(), index = df.index.values)
dtm_df

,abati[(meaning-unknown)]n,abbūtu[fatherhood]n,abiktu[defeat]n,abku[captive]n,abnu[stone]n,abu[father]n,abullu[(city)-gate]n,abāku[lead-away]v,abālu[dry-(up)]v,abāru[(the-metal)-lead]n,...,ṭābtu[salt]n,ṭābtānu[doer-of-good]n,ṭābtūtu[goodwill]n,ṭātu[bribe]n,ṭēhu[immediate-vicinity]n,ṭēmu[(fore)thought]n,ṭēmūtu[of-order]n,ṭīdu[clay]n,ṭīmu[yarn]n,ṭūbu[goodness]n
saao/saa01/P224485,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
saao/saa01/P313416,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
saao/saa01/P313417,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
saao/saa01/P313425,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
saao/saa01/P313427,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
saao/saa19/P393696,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
saao/saa19/P393702,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
saao/saa19/P393703,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
saao/saa19/P393709,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
dist = squareform(pdist(dtm_df, 'cosine'))

Compute the position of each document using tSNE. The variable `pos` holds the `x` and `y`  coordinates.

In [71]:
tsne = TSNE(n_components = 2, random_state=0, metric="precomputed")
X_tsne = tsne.fit_transform(dist)

In [72]:
tsne_x = [x for x, y in X_tsne]
tsne_y = [y for x, y in X_tsne]

Create lists of the most prevalent topic, the probability of the most prevalent topic, and the text name for each document. These lists are used in the Bokeh visualization.

In [73]:
prevalent_topic = list(d_t_df.idxmax(axis=1))
probability = [d_t_df.iloc[i][prevalent_topic[i]] for i in range(0, len(corpus))]
#probability takes the maximum value of each row - some better way of doing this?
text_name = list(df['text_name'])

In [74]:
len(prevalent_topic), len(probability), len(text_name)

(1754, 1754, 1754)

# Define Colors

Create a palette of the length `ntopics` and link that to topic numbers in the dictionary `colormap`. The list `colors` indicates the proper color for each document (determined by the most prevalent topic). 

In [75]:
colormap = {'topic_1': "orange", 'topic_2': "olive", 'topic_3': "firebrick", 
          'topic_4': "gold", 'topic_5': "red", 'topic_6': "fuchsia", 'topic_7': "green", 
          'topic_8': "blue", 'topic_9': "purple", 'topic_10': "aqua", 'topic_11': "yellow", 
          'topic_12': "indigo", 'topic_13': "blueviolet", 'topic_14': "beige", 'topic_15':"navy", 'topic_16': 'chocolate',
           'topic_17': 'azure', 'topic_18': 'coral', 'topic_19': 'crimson', 'topic_20': 'darkblue', 'topic_21': 'darkkhaki', 
            'topic_22': 'darkseagreen', 'topic_23': 'darkturquoise', 'topic_24': 'deeppink', 'topic_25': 'black'}

Import Bokeh and create data source. ColumnDataSource is a Bokeh function that creates a `source`, a database that holds information about each of the data points that are plotted. Bokeh can use the `source` to hold `x` and `y` coordinates; to select color, shape, or size; to include data or metadata in tooltips, to create a link to a web page, etc.

In [76]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool, CustomJS
from bokeh.models.widgets import Slider
from bokeh.plotting import figure, output_file, save, output_notebook, show
from bokeh.layouts import widgetbox, column

#output_notebook()

In [77]:
data=dict(
        x=tsne_x,
        y=tsne_y,
        id_text=list(d_t_df.index.values),
        size = probability/max(probability)*10,
        probability = probability,
        topic = prevalent_topic,
        text_name = text_name,
        color = [colormap[n] for n in prevalent_topic]
    )
title="Topic Distribution tSNE. Size of the circle represents prevalence of the topic"

Draw the visualization. The visualization provides various tools for further exploration:
- tooltips (provides topic, probability, text name and URL)
- box zoom
- wheel zoom
- pan
- reset
- link to document edition
- save the visualization

In addition, the visualization has two sliders that allow the user to select two topics.

In [78]:
from bokeh.models import CustomJS, ColumnDataSource, Slider, CDSView, CustomJSFilter
from bokeh.plotting import Figure, reset_output
source = ColumnDataSource(data)
plot = Figure(plot_width=600,
              plot_height=600,
              tools="tap,pan,wheel_zoom,box_zoom,reset,save", 
              title=title)
plot.add_tools(HoverTool(
        tooltips=[
            ("url", "http://oracc.org/" + "@id_text"),
            (("topic, probability"), ("@topic, @probability")),
            ("text name", "@text_name")
        ]
        ))
slider1 = Slider(start=0, end=ntopics, value=0, step=1, title="Topic 1")
slider2 = Slider(start=0, end=ntopics, value=0, step=1, title="Topic 2")
callback = CustomJS(args=dict(source=source, topic1 = slider1, 
                              topic2 = slider2, cm = colormap), code="""
    var data = source.data;
    topic = data['topic']
    for (i = 0; i < topic.length; i++) {
        if (topic[i].substring(6) == topic1.value) {
            data.color[i] = cm[topic[i]]
        } else if (topic1.value == 0) {
            data.color[i] = cm[topic[i]]
        } else {
            data.color[i] = 'grey'
        }
        if (topic[i].substring(6) == topic2.value) {
            data.color[i] = cm[topic[i]]
        } else if (topic2.value == 0) {
            data.color[i] = cm[topic[i]]
        }
    }
    source.change.emit();
""")

slider1.js_on_change('value', callback)
slider2.js_on_change('value', callback)

plot.circle('x', 
            'y', 
            color='color', 
            fill_alpha=.5, 
            size='size', 
            source=source) #, view=view)
plot.axis.visible = False

url = "http://oracc.museum.upenn.edu/@id_text"
taptool = plot.select(type=TapTool)
taptool.callback = OpenURL(url=url)

layout = column(slider1, slider2, plot)
try:
    reset_output()
    output_notebook()
    show(layout)  # angrily yells at me about single ownership
except:
    output_notebook()
    show(layout)  # ..
#show(layout)

Loading BokehJS ...